# Solving Equilibria with Complete Markets

Contents:

- [Solving Equilibria with Complete Markets](#Solving-Equilibria-with-Complete-Markets)  
  - [Key Equations](#Key-Equations)  
  - [Negishi Algorithm](#Negishi-Algorithm)
  - [Setting Up Complete Markets](#Setting-Up-Complete-Markets)  
  - [Steps 1 and 2 of the Algorithm](#Steps-1-and-2-of-the-Algorithm)   
  - [Step 3 of the Algorithm](#Step-3-of-the-Algorithm)   
  - [Implementing the Algorithm](#Implementing-the-Algorithm)   

This lab includes:

(1) An overview of the key equations necessary for solving for complete market equilibria;

(2) An overview of the Negishi algorithm;

(3) A basic setup for a model with complete markets;

(4) An application of a single iteration of steps 1 and 2 of the algorithm;

(5) An application of a single iteration of step 3 of the algorithm;

(6) An implementation of the full algorithm up to convergence.

----

## Key Equations

In this section I list all equations relevant to solving for an equilibrium of a complete markets model.

All equations are taken from Ljungqvist and Sargent, and numbered accordingly.

### Eq. (8.2.2)

A feasible allocation satisfies 

\begin{align*}
\sum_{i} c_t^i(s^t) \leq \sum_{i} y_t^i(s^t) \, ,
\end{align*}

for all $t$ and for all $s^t$.

### Eq. (8.5.1)

The consumer's budget constraint is expressed as 

\begin{align*}
\sum_{t=0}^\infty \sum_{s^t} q_t^0(s^t)c_t^i(s^t) \leq \sum_{t=0}^\infty \sum_{s^t} q_t^0(s^t) y_t^i(s^t)  \, .
\end{align*}

### Eq. (8.5.4)

The following equation will later be used to solve for $q_t^0(s^t)$:

\begin{align*}
\beta^t u_i' (c_t^i(s^t)) \pi_t(s^t) = \mu_i q_t^0(s^t) \, .
\end{align*}

### Eq. (8.5.5)

The following equation is obtained by dividing both sides of Eq. (8.5.4) by itself for a different $i$:

\begin{align*}
\frac{u_i'(c_t^i(s^t))}{u_j'(c_t^j(s^t))} = \frac{\mu_i}{\mu_j} \, ,
\end{align*}

for all pairs of $(i,j)$.

### Eq. (8.5.6)

An equilibrium allocation solves equations (8.2.2), (8.5.1), and (8.5.5).

Note that equation (8.5.5) implies 

\begin{align*}
c_t^i(s^t) = u_i'^{-1} \left\{ u_1' (c_t^1(s^t)) \frac{\mu_i}{\mu_1} \right\} \, .
\end{align*}

### Eq. (8.5.7)

Substituting (8.5.6) into Eq. (8.2.2) at equality gives 

\begin{align*}
\sum_i u_i'^{-1} \left\{ u_1' (c_t^1(s^t)) \frac{\mu_i}{\mu_1} \right\}  = \sum_i y_t^i(s^t) \, .
\end{align*}

## Negishi Algorithm

To compute an equilibrium, we have to determine ratios of the Lagrange multipliers, $\mu_i/\mu_1$, $i = 1,\ldots,I$, that appear in equations (8.5.6) and (8.5.7).

The following **Negishi algorithm** accomplishes this using the following steps:

### Step \#1

Fix any positive value for one $\mu_i$, say $\mu_1$, throughout the algorithm.
Guess positive values for the remaining $\mu_i$'s.
Then solve equations (8.5.6) and (8.5.7) for a candidate consumption allocation $c^i$, $i = 1,\ldots,I$.

### Step \#2

Use (8.5.4) for any consumer $i$ to solve for the price system $q_t^0(s^t)$.

### Step \#3

For $i = 1,\ldots,I$, check the budget constraint (8.5.1).
For those $i$'s for which the cost of consumption exceeds the value of their endowment, raise $\mu_i$, while for those $i$'s for which the reverse inequality holds, lower $\mu_i$.

### Step \#4

Iterate to convergence on steps 1-3.

## Setting Up Complete Markets

Load all necessary packages.

In [62]:
using LinearAlgebra, NLsolve

Specify basic model properties.

In [63]:
I = 3 # number of agents
μ = ones(I) # Lagrange multipliers for all agents
s = [1, 2] # states 
N = length(s) # total number of states 
W = zeros(N,I) # empty wage matrix 

for i in 1:N
    W[i,:] = randn(I) # Fill out wage matrix
end 

T = 3 # total number of time periods (t = 0,1,...,T)
;

Create a function that generates the set of all possible histories $s^t$ for any given $t$ for the specified model.

In [364]:
all_perm(s, t) = vec(map(collect, Iterators.product(ntuple(_ -> s, t+1)...)))

all_perm (generic function with 1 method)

Specify some more model primitives.

Specifically, set up the underlying Markov process for the state variable.

In [65]:
P0 = ones(N)/N # initial distribution
M = [0.9 0.1; 0.5 0.5] # stochastic matrix

2×2 Array{Float64,2}:
 0.9  0.1
 0.5  0.5

Create a function that returns the probability of any given history $s^t$.

In [66]:
function outcome_prob(history, P0, M)
    
# Store probability of initial state 
probability = P0[history[1]]

# Compute probability of `outcome` sequence
for i in 2:length(history)
    probability = probability * M[history[i-1], history[i]]
    end 
    
    # Return `probability` -- prob. of sequence `outcome`
    return probability
end 

outcome_prob (generic function with 1 method)

Let's make sure that our functions work.

In [366]:
history = all_perm(s, 2)

8-element Array{Array{Int64,1},1}:
 [1, 1, 1]
 [2, 1, 1]
 [1, 2, 1]
 [2, 2, 1]
 [1, 1, 2]
 [2, 1, 2]
 [1, 2, 2]
 [2, 2, 2]

In [68]:
history = all_perm(s, 2)[1]
outcome_prob(history, P0, M)

0.405

The above code is sufficient to allow us to set up any kind of state process.

Lastly, specify the consumers' utility functions.

Here we consider a constant relative risk-aversion (CRRA) utility function:

In [367]:
γ = 2 # specify γ parameter
u(c) = c^(-γ) # utility function 
u_inv(u) = u^(-1/γ) # inverse of the utility function

u_inv (generic function with 1 method)

Recall that the CRRA specification can be used to solve for an equilibrium analytically.

Lastly, let's just re-define all model primitives in one place for easy reference.

In [369]:
I = 3 # number of agents
μ = ones(I) # Lagrange multipliers for each agent
s = [1, 2] # states 
N = length(s) # total number of states 
W = zeros(N,I) # empty wage matrix 
β = 0.95

for i in 1:N
    W[i,:] = rand(1:10,I) # Fill out wage matrix
end 

T = 3 # total number of time periods (t = 0,1,...,T)

P0 = ones(N)/N # initial distribution
M = [0.9 0.1; 0.5 0.5] # stochastic matrix

γ = 0.5 # specify γ parameter
u(c) = c^(-γ) # utility function derivative 
u_inv(u) = u^(-1/γ) # inverse of the utility function derivative

μ = [1.0,1.0,1.0] # Lagrange multipliers for each agent
;

## Steps 1 and 2 of the Algorithm

In [370]:
"""
Steps #1 & #2 of the algorithm.

Compute prices and consumption associated with all s^t.

"""

# Create empty containers for 
# agent consumption, prices, probabilities, and income
C = zeros(T+1, length(all_perm(s,T)), I) # agent consumptions
Q = zeros(T+1, length(all_perm(s,T))) # prices
P = zeros(T+1, length(all_perm(s,T))) # probabilities
Y = zeros(T+1, length(all_perm(s,T)), I) # agent endowments

# Fill out all containers
for t in 0:T
    for st in 1:length(all_perm(s, t))

        # Store realization of s^t
        history = all_perm(s, t)[st]
            
        # Store final state of realized s^t
        n = history[end]
        
        # Store prob. of realized s^t
        prob = outcome_prob(history, P0, M)
        
        # Define system of equations
        # equivalent to Eqs. (8.5.6) and (8.5.7)
        g(cons) = [ sum([ u_inv(u(cons[1])*(μ[i]/μ[1])) - sum(W[n,:]) for i in 1:I]);
                        [cons[i] - u_inv(u(cons[1])*(μ[i]/μ[1])) for i in 2:I] ]
        
        # Solve system of equations
        solution = nlsolve(g, ones(I))
        c = solution.zero # solution for (c_t^1(s^t),...,c_t^I(s^t))
        
        # Solve for price system at t given s^t
        # using Eq. (8.5.4)
        #z(price) = (β^t)*u(c[1])*prob - μ[1]*price[1]
        #p_solution = nlsolve(z, [-15.0,15.0])
        price = (β^t)*u(c[2])*prob/μ[2]
        
        # Store consumption
        for i in 1:I
            C[t+1,st,i] = c[i]
        end 
        
        # Store price
        Q[t+1,st] = price
        
        # Store probability
        P[t+1,st] = prob
        
        # Store endowment
        for i in 1:I
            Y[t+1,st,i] = W[n,i]
        end 
        
    end 
end 

## Step 3 of the Algorithm

In [371]:
"""
Step #3 of the algorithm.

Check budget constraints and adjust μ.
"""

# Create empty vectors to store
# outflows and inflows for all agents
outflow = zeros(I)
inflow  = zeros(I)

for i in 2:I
    
    outflow[i] = sum([ sum([ Q[t+1,st]*C[t+1,st,i] for st in 1:length(all_perm(s, t)) ]) for t in 0:T])
    inflow[i] = sum([ sum([ Q[t+1,st]*Y[t+1,st,i] for st in 1:length(all_perm(s, t)) ]) for t in 0:T])
    
    if outflow[i] > inflow[i]
        μ[i] = μ[i]*(1.02)
    elseif outflow[i] < inflow[i]
        μ[i] = μ[i]*(0.95)
    end 
    
end 

@show outflow;
@show inflow;
@show inflow - outflow;
@show μ ;

outflow = [0.0, 17.24490473683121, 17.24490473683121]
inflow = [0.0, 3.671200809826186, 6.8650134985572215]
inflow - outflow = [0.0, -13.573703927005024, -10.379891238273988]
μ = [1.0, 1.02, 1.02]


In [375]:
Y

4×16×3 Array{Float64,3}:
[:, :, 1] =
 9.0  7.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 9.0  9.0  7.0  7.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 9.0  9.0  9.0  9.0  7.0  7.0  7.0  7.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 9.0  9.0  9.0  9.0  9.0  9.0  9.0  9.0     7.0  7.0  7.0  7.0  7.0  7.0  7.0

[:, :, 2] =
 4.0  6.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.0  4.0  6.0  6.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.0  4.0  4.0  4.0  6.0  6.0  6.0  6.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 4.0  4.0  4.0  4.0  4.0  4.0  4.0  4.0     6.0  6.0  6.0  6.0  6.0  6.0  6.0

[:, :, 3] =
 8.0  10.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 8.0   8.0  10.0  10.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
 8.0   8.0   8.0   8.0  10.0  10.0      0.0   0.0   0.0   0.0   0.0   0.0
 8.0   8.0   8.0   8.0   8.0   8.0     10.0  10.0  10.0  10.0  10.0  10.0

In [376]:
C

4×16×3 Array{Float64,3}:
[:, :, 1] =
 21.0  23.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  23.0  23.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  21.0  21.0  23.0  23.0      0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  21.0  21.0  21.0  21.0     23.0  23.0  23.0  23.0  23.0  23.0

[:, :, 2] =
 21.0  23.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  23.0  23.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  21.0  21.0  23.0  23.0      0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  21.0  21.0  21.0  21.0     23.0  23.0  23.0  23.0  23.0  23.0

[:, :, 3] =
 21.0  23.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  23.0  23.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  21.0  21.0  23.0  23.0      0.0   0.0   0.0   0.0   0.0   0.0
 21.0  21.0  21.0  21.0  21.0  21.0     23.0  23.0  23.0  23.0  23.0  23.0

In [377]:
P

4×16 Array{Float64,2}:
 0.5     0.5     0.0     0.0     0.0     …  0.0     0.0     0.0     0.0
 0.45    0.25    0.05    0.25    0.0        0.0     0.0     0.0     0.0
 0.405   0.225   0.025   0.125   0.045      0.0     0.0     0.0     0.0
 0.3645  0.2025  0.0225  0.1125  0.0225     0.0225  0.0125  0.0125  0.0625

In [379]:
Q

4×16 Array{Float64,2}:
 0.109109   0.104257   0.0         …  0.0         0.0         0.0
 0.0932881  0.0518267  0.00990443     0.0         0.0         0.0
 0.0797614  0.0443119  0.00492354     0.0         0.0         0.0
 0.068196   0.0378866  0.00420963     0.00223469  0.00223469  0.0111734

## Implementing the Algorithm

In [361]:
μ = [1.0,1.1,0.9]

for iter in 1:500

# Create empty containers for 
# agent consumption, prices, probabilities, and income
C = zeros(T+1, length(all_perm(s,T)), I) # agent consumptions
Q = zeros(T+1, length(all_perm(s,T))) # prices
P = zeros(T+1, length(all_perm(s,T))) # probabilities
Y = zeros(T+1, length(all_perm(s,T)), I) # agent incomes

# Fill out all containers
for t in 0:T
    for st in 1:length(all_perm(s, t))

        # Store realization of s^t
        history = all_perm(s, t)[st]
            
        # Store final state of realized s^t
        n = history[end]
        
        # Store prob. of realized s^t
        prob = outcome_prob(history, P0, M)
        
        # Define system of equations
        # equivalent to Eqs. (8.5.6) and (8.5.7)
        g(cons) = [sum([ u_inv(u(cons[1])*(μ[i]/μ[1])) - sum(W[n,:]) for i in 1:I]);
                        [cons[i] - u_inv(u(cons[1])*(μ[i]/μ[1])) for i in 2:I]]
        
        # Solve system of equations
        solution = nlsolve(g, ones(I))
        c = solution.zero # solution for (c_t^1(s^t),...,c_t^I(s^t))
        
        # Solve for price system at t given s^t
        # using Eq. (8.5.4)
        #z(price) = (β^t)*u(c[1])*prob - μ[1]*price[1]
        #p_solution = nlsolve(z, [-15.0,15.0])
        price = (β^t)*u(c[2])*prob/μ[2]
        
        # Store consumption
        for i in 1:I
            C[t+1,st,i] = c[i]
        end 
        
        # Store price
        Q[t+1,st] = price
        
        # Store probability
        P[t+1,st] = prob
        
        # Store income
        for i in 1:I
            Y[t+1,st,i] = W[n,i]
        end 
        
    end 
end 

# Create empty vectors to store
# outflows and inflows for all agents
outflow = zeros(I)
inflow  = zeros(I)

for i in 2:I
    
    outflow[i] = sum([ sum([ Q[t+1,st]*C[t+1,st,i] for st in 1:length(all_perm(s, t)) ]) for t in 0:T])
    inflow[i] = sum([ sum([ Q[t+1,st]*Y[t+1,st,i] for st in 1:length(all_perm(s, t)) ]) for t in 0:T])
    
    if outflow[i] > inflow[i]
        μ[i] = μ[i]*(1.02)
    elseif outflow[i] < inflow[i]
        μ[i] = μ[i]*(0.95)
    end 
    
end 
    
end 

In [363]:
μ

3-element Array{Float64,1}:
 1.0
 2.631204626094999
 2.6646130328716